# Pluggable Web Scraper and Summarizer with Interface-Based Design

This system implements a **pluggable architecture** for web scraping and summarization, built on interface-based design using Python’s `Protocol` types. Each stage of the pipeline—content fetching, HTML parsing, and LLM-based summarization—is defined through explicit structural contracts rather than concrete implementations. Components like `RequestsFetcher`, `RobustSoupParser`, and `OllamaClient` fulfill these protocols and can be swapped independently, enabling flexibility, testing, and future extension without modifying core logic. Immutable data models (`@dataclass(frozen=True)`) enforce data integrity throughout the pipeline, while the design cleanly separates concerns across modules to support maintainability and modular growth.

In [0]:
from dataclasses import dataclass
from typing import Protocol, Optional, List, Dict, Tuple
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
import logging
import chardet

# Configuration

In [0]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
}
DEFAULT_TIMEOUT = 10
UNWANTED_TAGS = ["script", "style", "nav", "header", "footer", "img", "input"]

# Data Models

In [0]:
@dataclass(frozen=True)
class RawResponse:
    content: bytes
    status_code: int
    encoding: str
    headers: Dict[str, str]
    elapsed: float
    final_url: str

@dataclass(frozen=True)
class WebsiteContent:
    url: str
    title: str
    text: str
    status_code: int
    response_time: float

@dataclass(frozen=True)
class LLMResponse:
    content: str
    model: str
    tokens_used: int

# Protocols

In [0]:
class ContentFetcher(Protocol):
    def fetch(self, url: str) -> RawResponse: ...

class ContentParser(Protocol):
    def parse(self, response: RawResponse) -> WebsiteContent: ...

class LLMClient(Protocol):
    def generate(self, messages: List[Dict[str, str]], model: str) -> LLMResponse: ...


# Implementations

In [0]:
class RequestsFetcher:
    def __init__(self, 
                 headers: Dict[str, str] = HEADERS,
                 timeout: int = DEFAULT_TIMEOUT,
                 max_redirects: int = 5):
        self.headers = headers
        self.timeout = timeout
        self.max_redirects = max_redirects

    def fetch(self, url: str) -> RawResponse:
        logger.info(f"Fetching content from {url}")
        try:
            response = requests.get(
                url,
                headers=self.headers,
                timeout=self.timeout,
                allow_redirects=True,
                stream=False  # Prevent partial content issues
            )
            response.raise_for_status()
            
            return RawResponse(
                content=response.content,
                status_code=response.status_code,
                encoding=response.encoding,
                headers=dict(response.headers),
                elapsed=response.elapsed.total_seconds(),
                final_url=response.url
            )
        except requests.exceptions.RequestException as e:
            logger.error(f"Failed to fetch {url}: {str(e)}")
            raise

class RobustSoupParser:
    def __init__(self, unwanted_tags: Tuple[str] = UNWANTED_TAGS):
        self.unwanted_tags = unwanted_tags

    def parse(self, response: RawResponse) -> WebsiteContent:
        logger.info(f"Parsing content from {response.final_url}")
        
        # Detect encoding if not provided
        encoding = response.encoding or self._detect_encoding(response.content)
        
        try:
            decoded_content = response.content.decode(encoding, errors='replace')
            soup = BeautifulSoup(decoded_content, 'html.parser')
        except Exception as e:
            logger.error(f"Failed to parse content: {str(e)}")
            raise

        return WebsiteContent(
            url=response.final_url,
            title=self._extract_title(soup),
            text=self._clean_content(soup),
            status_code=response.status_code,
            response_time=response.elapsed
        )

    def _detect_encoding(self, content: bytes) -> str:
        result = chardet.detect(content)
        return result['encoding'] or 'utf-8'

    def _extract_title(self, soup: BeautifulSoup) -> str:
        title_tag = soup.find('title')
        return title_tag.text.strip() if title_tag else "Untitled"

    def _clean_content(self, soup: BeautifulSoup) -> str:
        # Remove unwanted tags
        for tag in self.unwanted_tags:
            for element in soup.find_all(tag):
                element.decompose()

        # Extract text with semantic line breaks
        text = '\n\n'.join([
            element.get_text().strip()
            for element in soup.find_all(['p', 'h1', 'h2', 'h3', 'article'])
            if element.get_text().strip()
        ])
        
        return text or "No readable content found"

class OllamaClient:
    def __init__(self, 
                 base_url: str = 'http://localhost:11434/v1',
                 api_key: str = 'ollama',
                 max_retries: int = 3):
        self.client = OpenAI(base_url=base_url, api_key=api_key)
        self.max_retries = max_retries

    def generate(self, 
                messages: List[Dict[str, str]], 
                model: str = "llama3.2") -> LLMResponse:
        logger.info(f"Generating summary with {model}")
        
        for attempt in range(self.max_retries):
            try:
                response = self.client.chat.completions.create(
                    model=model,
                    messages=messages
                )
                return LLMResponse(
                    content=response.choices[0].message.content,
                    model=model,
                    tokens_used=response.usage.total_tokens
                )
            except Exception as e:
                if attempt == self.max_retries - 1:
                    logger.error(f"Failed after {self.max_retries} attempts: {str(e)}")
                    raise
                logger.warning(f"Retry {attempt + 1}/{self.max_retries}")

# Core Pipeline

In [0]:
class SummarizationPipeline:
    SYSTEM_PROMPT = """You are a professional web content analyst. Provide a structured markdown summary containing:
- Key points
- Notable statistics
- Important names/dates
- Actionable insights
Avoid navigation content and marketing fluff."""

    def __init__(self,
                fetcher: ContentFetcher,
                parser: ContentParser,
                llm_client: LLMClient):
        self.fetcher = fetcher
        self.parser = parser
        self.llm_client = llm_client

    def summarize(self, url: str, model: str = "llama3.2") -> LLMResponse:
        raw_response = self.fetcher.fetch(url)
        website_content = self.parser.parse(raw_response)
        messages = self._build_messages(website_content)
        return self.llm_client.generate(messages, model)

    def _build_messages(self, content: WebsiteContent) -> List[Dict[str, str]]:
        user_prompt = f"""**Website Analysis Request**
URL: {content.url}
Title: {content.title}

Content:
{content.text[:8000]}  # Truncate to stay within context window

Please provide a comprehensive summary following the guidelines above."""
        
        return [
            {"role": "system", "content": self.SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt}
        ]

# Factory & Presentation

In [0]:
def create_default_pipeline() -> SummarizationPipeline:
    return SummarizationPipeline(
        fetcher=RequestsFetcher(),
        parser=RobustSoupParser(),
        llm_client=OllamaClient()
    )

class JupyterPresenter:
    @staticmethod
    def display(response: LLMResponse) -> None:
        display(Markdown(f"""
## Summary Results
**Model**: {response.model}  
**Tokens Used**: {response.tokens_used}  
**Summary**:
{response.content}
        """))
        

# Execution

In [0]:
pipeline = create_default_pipeline()
try:
    response = pipeline.summarize("https://edwarddonner.com")
    JupyterPresenter.display(response)
except Exception as e:
    logger.error(f"Summarization failed: {str(e)}")
    display(Markdown("## Error\nUnable to generate summary. Please check the URL and try again."))